<div style="background-color:	#103060; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white">Website Automatisering met WP-Admin </h1>
<b style="color:white">Rogier Gernaat</b>
    </div>

<a name='start'></a>

<div style="background-color:#006069; color:white; padding: 10px;">
    <strong>Inladen van modules en library's</strong>
</div>


In [4]:
!pip install python-wordpress-xmlrpc
!pip install mysql-connector-python
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install numpy
!pip install python-dotenv
!pip install requests
!pip install dotenv





  Using cached python-wordpress-xmlrpc-2.3.zip (19 kB)
Using legacy 'setup.py install' for python-wordpress-xmlrpc, since package 'wheel' is not installed.
    Running setup.py install for python-wordpress-xmlrpc: started
    Running setup.py install for python-wordpress-xmlrpc: finished with status 'done'


You should consider upgrading via the 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


You should consider upgrading via the 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe -m pip install --upgrade pip' command.


...

You should consider upgrading via the 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)


You should consider upgrading via the 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached dotenv-0.0.5.tar.gz (2.4 kB)


    ERROR: Command errored out with exit status 1:
     command: 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\rogie\\AppData\\Local\\Temp\\pip-install-b1zj4q7r\\dotenv\\setup.py'"'"'; __file__='"'"'C:\\Users\\rogie\\AppData\\Local\\Temp\\pip-install-b1zj4q7r\\dotenv\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\rogie\AppData\Local\Temp\pip-pip-egg-info-v_j5iwue'
         cwd: C:\Users\rogie\AppData\Local\Temp\pip-install-b1zj4q7r\dotenv\
    Complete output (52 lines):
        ERROR: Command errored out with exit status 1:
         command: 'c:\users\rogie\onedrive\bureaublad\git-clone-stage\stage-map\myvenv\scripts\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\rogie\\AppData\\Local\\T

In [5]:
import os
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
from wordpress_xmlrpc import Client
from wordpress_xmlrpc.methods.posts import EditPost

<div style="background-color:#006069; color:white; padding: 10px;">
    <strong>Inladen de .env bestanden</strong>
</div>
Dit laad de gegevens in om in te loggen in de database en wp-admin

In [6]:
# Laad Azure-configuratie uit .env
load_dotenv()
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_ssl_ca = os.getenv('DB_SSL_CA')

# WordPress-credentials
wp_url = "https://samenzoetermeergezond.nl/xmlrpc.php"
wp_user = "SamenZoeterMeerGezond"
wp_password = "$Lmzk#S1U6gvM5p8&@@Gn2NE"

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>SQL Verbinding Class </strong></h2>
</div>


In [7]:
class AzureDBConnector:
    def __init__(self, host, port, user, password, database, ssl_ca):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.ssl_ca = ssl_ca
        self.connection = None

    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                port=self.port,
                user=self.user,
                password=self.password,
                database=self.database,
                ssl_ca=self.ssl_ca
            )
            print("Verbonden met de Azure Database.")
        except mysql.connector.Error as err:
            print(f"Fout bij het verbinden met de database: {err}")

    def fetch_query_data(self, query):
        if not self.connection:
            print("Geen actieve databaseverbinding.")
            return pd.DataFrame()
        try:
            cursor = self.connection.cursor(dictionary=True)
            cursor.execute(query)
            rows = cursor.fetchall()
            df = pd.DataFrame(rows)
            return df
        except Exception as err:
            print(f"Fout bij het uitvoeren van de query: {err}")
            return pd.DataFrame()
        finally:
            cursor.close()

    def close(self):
        if self.connection:
            self.connection.close()
            print("Verbinding met de database gesloten.")

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Layout maken van de pagina's </strong></h2>
</div>


In [147]:
class ActivityPublisher:
    def __init__(self, wp_url, wp_user, wp_password):
        self.client = Client(wp_url, wp_user, wp_password)

    def build_activities_html(self, activities, column_mapping, page_id):
        """
        Bouw de HTML voor de activiteitenkaarten.
        """
        html = """
        <div class="activities-container">
        """
        default_image_url = "https://samenzoetermeergezond.nl/wp-content/uploads/2024/12/Jaaroverzicht-Samen-ZoeterMeer-Gezond-2024-1.png"

        for _, row in activities.iterrows():
            full_desc = row.get(column_mapping.get('beschrijving', ''), '') or ''
            shortened_desc = (full_desc[:150] + '...') if len(full_desc) > 150 else full_desc

            title = row.get(column_mapping.get('titel', 'Onbekende titel'), 'Onbekende titel')
            location = row.get(column_mapping.get('locatie', ''), '')
            date = row.get(column_mapping.get('datum', ''), '')
            link = row.get(column_mapping.get('link', '#'), '#')

            # Logica voor afbeelding selectie
            image_url = row.get(column_mapping.get('url_afbeelding', ''), None)
            if not image_url:
                image_url = row.get(column_mapping.get('url_header_afbeelding', ''), None)
            if not image_url:
                image_url = default_image_url

            alt_text = f"Afbeelding van {title}"

            # Logica voor locatie en datum weergave met een dun lijntje
            location_date_html = ""
            if location and date:
                location_date_html = f"""
                <hr style="border: 0; border-top: 1px solid rgba(0, 0, 0, 0.1); margin: 8px 0;">
                <p class='activity-location' style='font-size: 0.85em; color: #777; margin: 8px 0;'>
                    <strong>Locatie:</strong> {location} <br>
                    <strong>Datum:</strong> {date}
                </p>
                """
            elif location:
                location_date_html = f"""
                <hr style="border: 0; border-top: 1px solid rgba(0, 0, 0, 0.1); margin: 8px 0;">
                <p class='activity-location' style='font-size: 0.85em; color: #777; margin: 8px 0;'>
                    <strong>Locatie:</strong> {location}
                </p>
                """
            elif date:
                location_date_html = f"""
                <hr style="border: 0; border-top: 1px solid rgba(0, 0, 0, 0.1); margin: 8px 0;">
                <p class='activity-location' style='font-size: 0.85em; color: #777; margin: 8px 0;'>
                    <strong>Datum:</strong> {date}
                </p>
                """

            card_html = f"""
            <div class="activity-card" style="background-color: #fff; color: #000; border-radius: 8px; overflow: hidden; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
                <img src="{image_url}" alt="{alt_text}" loading="lazy" style="width: 100%; height: auto; object-fit: cover; aspect-ratio: 16/9;">
                <div class="activity-content" style="padding: 16px;">
                    <h3 class="activity-title" style="font-size: 1.2em; margin-bottom: 8px;">{title}</h3>
                    <p class="activity-desc" style="font-size: 0.9em; color: #555; margin-bottom: 8px;">{shortened_desc}</p>
                    {location_date_html}
                    <a href="{link}" class="activity-link" style="display: inline-block; padding: 8px 12px; background-color: #007BFF; color: #fff; text-decoration: none; border-radius: 4px;">Meer informatie</a>
                </div>
            </div>
            """
            html += card_html
        html += "</div>"

        # Voeg specifieke class toe voor pagina's behalve 8074
        if page_id != 8074:
            html = html.replace("activities-container", "activities-container other-pages")

        return html

    def build_sidebar_html(self, sidebar_links):
        """
        Bouwt de sidebar met klikbare blokken, maar logo boven de tekst (vertical stacking).
        """
        html = """
        <div class="sidebar-container">
            <h3 class="sidebar-title">Andere pagina's</h3>
            <div class="sidebar-list">
        """
        for item in sidebar_links:
            name = item["name"]
            url = item["url"]
            img_url = item["img"]
            html += f"""
            <div class="sidebar-item">
                <a href="{url}" target="_blank" rel="noopener" class="sidebar-link">
                    <div class="sidebar-logo">
                        <img src="{img_url}" alt="{name}" />
                    </div>
                    <div class="sidebar-name">{name}</div>
                </a>
            </div>
            """
        html += """
            </div>
        </div>
        """
        return html

    def combine_content_with_sidebar(self, activities_html, sidebar_html):
        """
        Volledige layout:
        - Links (flex:3) de activiteiten
        - Rechts (vast 300px) de sidebar
        - Body met blauwe achtergrond
        """
        background_url = "https://samenzoetermeergezond.nl/wp-content/uploads/2024/07/Achtergrond-blauw.jpg"

        full_html = f"""
        <html>
        <head>
            <style>
                body {{
                    margin: 0;
                    padding: 0;
                    background: url("{background_url}") no-repeat center center fixed;
                    background-size: cover;
                    font-family: "astounder-squared-lc-bb", Sans-serif;
                    color: #fff;
                }}
                .page-wrapper {{
                    display: flex;
                    justify-content: space-between;
                    align-items: flex-start;
                }}
                .main-content {{
                    flex: 3;
                    padding: 20px;
                }}
                .right-sidebar {{
                    width: 300px;
                    background-color: rgba(255, 255, 255, 0.8);
                    min-height: 100vh;
                    padding: 20px;
                    box-sizing: border-box;
                    color: #000;
                }}
                .activities-container {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 16px;
                    justify-content: center;
                }}
                .activities-container.other-pages {{
                    grid-template-columns: repeat(4, 1fr);
                }}
                .activity-card {{
                    width: 100%;
                    background-color: #ffffff;
                    color: #000000;
                    border: 2px solid #ffffff;
                    border-radius: 8px;
                    box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
                    overflow: hidden;
                    text-align: center;
                    font-family: inherit; 
                }}
                .activity-card img {{
                    width: 100%;
                    height: auto;
                    object-fit: cover;
                    aspect-ratio: 16/9;
                }}
                .activity-content {{
                    padding: 16px;
                }}
                .activity-title {{
                    font-size: 1.2em;
                    margin-bottom: 8px;
                    color: #000;
                }}
                .activity-desc {{
                    display: -webkit-box;
                    -webkit-line-clamp: 3;
                    -webkit-box-orient: vertical;
                    overflow: hidden;
                    text-overflow: ellipsis;
                    font-size: 0.9em;
                    color: #555;
                    margin-bottom: 8px;
                }}
                .activity-location {{
                    font-size: 0.8em;
                    color: #777;
                    margin-bottom: 16px;
                }}
                .activity-link {{
                    display: inline-block;
                    padding: 6px 10px;
                    background-color: #007BFF !important;
                    color: #ffffff !important;
                    text-decoration: none;
                    border-radius: 4px;
                    font-size: 0.85em;
                    font-family: inherit; 
                    transition: background-color 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
                }}
                .activity-link:hover {{
                    background-color: #0056b3 !important;
                    color: #ffffff !important;
                    box-shadow: 0 0 6px rgba(0, 123, 255, 0.7);
                }}
                .sidebar-container {{
                    width: 100%;
                }}
                .sidebar-title {{
                    font-size: 1.2em;
                    margin-bottom: 16px;
                    font-weight: bold;
                }}
                .sidebar-list {{
                    display: flex;
                    flex-direction: column;
                    gap: 10px;
                }}
                .sidebar-item {{
                    background: #fff;
                    color: #000;
                    border: 2px solid #cce;
                    border-radius: 4px;
                    padding: 4px;
                    width: 100%;
                    box-sizing: border-box;
                    transition: box-shadow 0.2s ease-in-out;
                    text-align: center; 
                }}
                .sidebar-item:hover {{
                    background-color: #f0f0f0;
                    box-shadow: 0 0 6px rgba(0, 123, 255, 0.7);
                }}
                .sidebar-link {{
                    display: flex;
                    flex-direction: column;
                    align-items: center;
                    text-decoration: none; 
                    color: inherit;
                }}
                .sidebar-logo {{
                    width: 100%;
                    height: 60px;
                    overflow: hidden;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    margin-bottom: 8px;
                }}
                .sidebar-logo img {{
                    width: 80%;
                    height: 100%;
                    object-fit: contain;
                }}
                .sidebar-name {{
                    font-weight: 600;
                    font-size: 0.9em;
                }}
            </style>
        </head>
        <body>
            <div class="page-wrapper">
                <div class="main-content">
                    {activities_html}
                </div>
                <div class="right-sidebar">
                    {sidebar_html}
                </div>
            </div>
        </body>
        </html>
        """
        return full_html

    def update_page_with_sidebar(self, page_id, activities, sidebar_links, column_mapping):
        """
        Update de WordPress-pagina met activiteiten en zijbalk.
        """
        try:
            activities_html = self.build_activities_html(activities, column_mapping, page_id)
            sidebar_html = self.build_sidebar_html(sidebar_links)
            final_html = self.combine_content_with_sidebar(activities_html, sidebar_html)

            page = {"post_content": final_html}
            self.client.call(EditPost(page_id, page))

            print(f"Pagina met ID {page_id} succesvol geüpdatet.")
        except Exception as err:
            print(f"Fout bij het updaten van de pagina: {err}")


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Inzet in de wijk </strong></h2>
</div>
Dit is code voor het invullen van de inzet in de wijk activiteiten op de correcte pagina

In [148]:
if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Dynamische kolomnamen per pagina
    page_column_mappings = {
        8074: {  # Voorbeeld pagina-ID
            'titel': 'Titel',
            'beschrijving': 'Extra_beschrijving',
            'locatie': 'Locatie',
            'datum': 'Datum_text',
            'link': 'Link',
            'url_afbeelding': 'URL_afbeelding'
        }
    }

    # 2. Haal activiteiten op
    page_id = 8074  # Pas dit aan voor de juiste pagina-ID
    column_mapping = page_column_mappings.get(page_id, {})
    query = "SELECT * FROM activiteiten"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "ZoeterMeerActief",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ZMA-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/ZMA-e1737116617322.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    publisher.update_page_with_sidebar(page_id, activiteiten, sidebar_links, column_mapping)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Pagina met ID 8074 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>ZoeterMeerActief </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [149]:
import time

if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Dynamische kolomnamen per pagina
    page_column_mappings = {
        8142: {  # Nieuwe pagina-ID voor de website
            'titel': 'Titel',
            'beschrijving': 'Extra_beschrijving',
            'datum': 'Startdatum',
            'link': 'Link',
            'url_afbeelding': 'URL_afbeelding',
            'url_header_afbeelding': 'Url_header_afbeelding'
        }
    }

    # 2. Haal activiteiten op
    page_id = 8142  # Nieuwe pagina-ID
    column_mapping = page_column_mappings.get(page_id, {})
    query = "SELECT Titel, Link, Extra_beschrijving, Startdatum, URL_afbeelding, Url_header_afbeelding FROM activiteiten_zma"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "Inzet In De Wijk",
            "url": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Screenshot-2025-01-17-171948.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Wacht 20 seconden voordat de update wordt uitgevoerd
    print("Wachten 20 seconden om serverbelasting te minimaliseren...")
    time.sleep(20)

    # Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    publisher.update_page_with_sidebar(page_id, activiteiten, sidebar_links, column_mapping)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Wachten 20 seconden om serverbelasting te minimaliseren...
Pagina met ID 8142 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Evie.nl </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [156]:
import time

if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Dynamische kolomnamen per pagina
    page_column_mappings = {
        8203: {  # Pagina-ID voor Evie.nl
            'titel': 'Titel',
            'beschrijving': 'Beschrijving',
            'link': 'Link',
            'url_afbeelding': 'Afbeelding_url',
        }
    }

    # 2. Haal activiteiten op
    page_id = 8203  # Pagina-ID voor Evie.nl
    column_mapping = page_column_mappings.get(page_id, {})
    query = "SELECT Titel, Link, Beschrijving, Afbeelding_url FROM evie_data"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "Inzet In De Wijk",
            "url": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Screenshot-2025-01-17-171948.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Wacht 20 seconden voordat de update wordt uitgevoerd
    print("Wachten 20 seconden om serverbelasting te minimaliseren...")
    time.sleep(20)

    # Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    publisher.update_page_with_sidebar(page_id, activiteiten, sidebar_links, column_mapping)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Wachten 20 seconden om serverbelasting te minimaliseren...
Pagina met ID 8203 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>GGD_Appstore </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [ ]:
import time

if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Dynamische kolomnamen per pagina
    page_column_mappings = {
        8203: {  # Pagina-ID voor Evie.nl
            'titel': 'Titel',
            'beschrijving': 'Beschrijving',
            'link': 'Link',
            'url_afbeelding': 'Afbeelding_url',
        }
    }

    # 2. Haal activiteiten op
    page_id = 8203  # Pagina-ID voor Evie.nl
    column_mapping = page_column_mappings.get(page_id, {})
    query = "SELECT Titel, Link, Beschrijving, Afbeelding_url FROM evie_data"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "Inzet In De Wijk",
            "url": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Screenshot-2025-01-17-171948.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "Nederland zorgt voor elkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/NLZVE-e1737116948577.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Wacht 20 seconden voordat de update wordt uitgevoerd
    print("Wachten 20 seconden om serverbelasting te minimaliseren...")
    time.sleep(20)

    # Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    publisher.update_page_with_sidebar(page_id, activiteiten, sidebar_links, column_mapping)

    # 6. Sluit de databaseverbinding
    db_connector.close()


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Nederland Zorgt Voor Elkaar </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [154]:
import time

if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Dynamische kolomnamen per pagina
    page_column_mappings = {
        8195: {  # Pagina-ID voor NLZVE
            'titel': 'Titel',
            'beschrijving': 'beschrijving_lang',
            'link': 'Link',
            'url_afbeelding': 'Afbeelding_url',
            'locatie': 'Locatie',
            'datum': 'Eindtijd',
        }
    }

    # 2. Haal activiteiten op
    page_id = 8195  # Pagina-ID voor NLZVE
    column_mapping = page_column_mappings.get(page_id, {})
    query = "SELECT Titel, Link, beschrijving_lang, Afbeelding_url, Locatie FROM activiteiten_nlzve"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "Inzet In De Wijk",
            "url": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Screenshot-2025-01-17-171948.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "ZoeterMeerActief",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ZMA-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/ZMA-e1737116617322.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Wacht 20 seconden voordat de update wordt uitgevoerd
    print("Wachten 20 seconden om serverbelasting te minimaliseren...")
    time.sleep(20)

    # Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    publisher.update_page_with_sidebar(page_id, activiteiten, sidebar_links, column_mapping)

    # 6. Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Wachten 20 seconden om serverbelasting te minimaliseren...
Pagina met ID 8195 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Scheidingspunt </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [ ]:
if __name__ == "__main__":
    # 1. Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # 2. Haal activiteiten op
    query = "SELECT titel, link, wat, wanneer FROM activiteiten_nlzve"
    activiteiten = db_connector.fetch_query_data(query)

    # 3. Sidebar-links (klikbaar)
    sidebar_links = [
        {
            "name": "Inzet In De Wijk",
            "url": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Screenshot-2025-01-17-171948.png"
        },
        {
            "name": "Evie.nl",
            "url": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Evie.nl_-e1737116746677.png"
        },
        {
            "name": "GGD appstore",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/GGDAPPSTORE.jpg"
        },
        {
            "name": "ZoeterMeerActief",
            "url": "https://samenzoetermeergezond.nl/testomgeving-ZMA-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/ZMA-e1737116617322.png"
        },
        {
            "name": "Scheidingspunt",
            "url": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Scheidingspunt-e1737116711397.png"
        },
        {
            "name": "UITgenda",
            "url": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/UITgenda-e1737116828219.png"
        },
        {
            "name": "ZoetermeerTegenEenzaamheid",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeertegeneenzaamheid-e1737116787436.png"
        },
        {
            "name": "ZoetermeerVoorElkaar",
            "url": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/",
            "img": "https://samenzoetermeergezond.nl/wp-content/uploads/2025/01/Zoetermeervoorelkaar-e1737116649533.png"
        }
    ]

    # 4. Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # 5. Update de pagina (deze functie krijgt nu ook sidebar_links mee)
    private_page_id = 8195  # Pas aan met de juiste pagina-ID
    publisher.update_page_with_sidebar(private_page_id, activiteiten, sidebar_links)

    # 6. Sluit de databaseverbinding
    db_connector.close()


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>UITgenda </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>ZoeterMeerTegenEenzaamHeid </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>ZoetermeerVoorElkaar </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina